Given two numbers: 'left' and 'right' (1 <= 'left' <= 'right' <= 200000000000000) return sum of all '1' occurencies in binary representations of numbers between 'left' and 'right' (including both)

Example:
countOnes 4 7 should return 8, because:
4(dec) = 100(bin), which adds 1 to the result.
5(dec) = 101(bin), which adds 2 to the result.
6(dec) = 110(bin), which adds 2 to the result.
7(dec) = 111(bin), which adds 3 to the result.
So finally result equals 8.

WARNING: Segment may contain billion elements, to pass this kata, your solution cannot iterate through all numbers in the segment!

In [255]:
def bin_same_length(left, right):
    # if left and right are the same, simply count the ones in the binary equivalent
    if left == right:
        return bin(left)[2:].count('1'), -1, -1

    # if length of binary is 1, there is 1 one
    len_bin = len(bin(left)[2:])
    if len_bin == 1:
        return 1, -1, -1

    # if left is at the bottom of the range for digits of this length, and right is at the top 
    # of the range, there's a short-cut to count the ones in all numbers within this range
    if 2**(len_bin-1) == left and (2**len_bin)-1 == right:
        # we know all numbers in the range start with a 1
        # for all combinations the remaining digits (i.e. n = len -1), the number of combinations 
        # with a given number of ones i is given by n!/i!(n-i)! combinations
        # multiply that by the number of ones to give n!/(i-1)!(n-i)! ones
        # then add 1 for each possible combination because we know the numbers in the range all start with 1
        from math import factorial
        result = 0
        for i in range(1, len_bin):
            result += factorial(len_bin-1) / (factorial(i-1) * factorial(len_bin-1-i))
        result += 2**(len_bin-1)
        return int(result), -1, -1

    # otherwise calculate the number of numbers in the range
    num_numbers = right - left + 1

    # convert both numbers to binary and find out how many digits they have in common at the start
    bin_left = bin(left)[2:]
    bin_right = bin(right)[2:]
    digits_in_common = ''
    for i in range(len(bin_left)):
        if bin_left[i] == bin_right[i]:
            digits_in_common += bin_left[i]
        else:
            break

    # determine the number of ones represented by these digits in common
    result = digits_in_common.count('1') * num_numbers

    # determine the numbers remaining after removing the prefix in common
    if digits_in_common == bin_left:
        lower = 0
    else:
        lower = int(bin_left[len(digits_in_common):],2)
    upper = int(bin_right[len(digits_in_common):],2)
    if lower > upper:
        lower, upper = upper, lower
    return result, lower, upper

def split_range(left, right):
    # convert left and right into binary 
    bin_left = bin(left)[2:]
    bin_right = bin(right)[2:]

    # check if all entries have the same length
    if len(bin_left) == len(bin_right):
        return [(left, right)]

    # split into multiple entries with the same length
    # first a range of numbers with the same length as left
    num_split = []
    num_split.append( (left, (2**len(bin_left))-1) )

    # then add ranges of numbers for each length of number between that of left and that of right
    if len(bin_right) - len(bin_left) > 1:
        for i in range( len(bin_left) + 1, len(bin_right) ):
            num_split.append( (2**(i-1), (2**i)-1) )

    # then add range of numbers with the same length as right
    if len(bin_right) > len(bin_left):
        num_split.append( ((2**(len(bin_right)-1)), right) )
    return num_split

def process_split(num_split):
    result = 0
    for entry in num_split:
        lower = entry[0]
        upper = entry[1]
        while lower > -1:
            num_ones, lower, upper = bin_same_length(lower, upper)
            result += num_ones
            if lower > -1:
                inner_num_split = split_range(lower, upper)
                result += process_split(inner_num_split)
                break
    return result

def countOnes(left, right):
    return process_split(split_range(left, right))

In [256]:
countOnes(173624870784686,239652917223111)
# expected_result = 1611203953924013

1611203953924013